

<center><img src="images/MLU-NEW-logo.png" alt="drawing" width="400" style="background-color:white; padding:1em;" /></center> <br/>

# <a name="0">Using Bedrock Agents with Boto3 SDK </a>
## <a name="0"> Lab 2a: Text to SQL (Simple) - without Knowledge Bases using Transactional Retrieval Mechanism</a>

## Lab Overview

In this lab, we will recreate lab 1 but with the Boto 3 SDK API. We will learn to use Bedrock Agents with Knowledge Bases invoked via Lambdas using ReAct Prompting. Internally Bedrock Agents automate prompt engineering and orchestration of user-requested tasks.

##### Notebook Kernel
Please choose `conda_python3` as the kernel type of the top right corner of the notebook if that does not appear by default.


<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>This lab automatically cleans up resources to be frugal. </h4>
    You can visit this section (<a href="#9"> Clean-up Resources</a>) to change the setting if you need to experiment with prompts and settings. Please run clean-up resources after you are done with experiments. <br/>
</div>
<br/>

## LLM Used
Anthropic Claude 3 Haiku 

## Use-case Overview
Here, we are setting up a retail customer service agent that helps customers purchase shoes via natural language conversations. User requests are broken down into multiple steps, with each step invoking APIs to fulfill the sub-tasks.

With ReAct, the sequence of actions for Agents follows a question-thought-action-observation paradigm:

    - The question is the user-requested task or problem to solve.
    - The thought is a reasoning step that helps demonstrate to the FM how to tackle the problem and identify an action to take.
    - The action is an API that the model can invoke from an allowed set of APIs.
    - The observation is the result of carrying out the action.
    

#### Chat window of user-agent conversations: 
<br/> <center><img src="images/retail-bot-workflow.png" alt="conversation between user and agent to buy a pair of shoes after validating user information. The bot gets customer information by asking the customer for their name, and then gets their details. Then the bot uses those details to find the right kinds of shoes for them, then checks its inventory database to be sure they're in stock, then creates a prompt that asks the FM to generate a response to the customer. Then the bot places an order on behalf of a customer." height="700" width="700" style="background-color:white; padding:1em;" /></center> <br/>

#### Sequence Diagram of user-agent conversations 
<br/> <center><img src="images/retail-flow-agents.png" alt="This figure is to show the sequence diagram to capture user and agent interaction for every conversation in the chat session to buy a pair of shoes. The bot gets customer information by asking the customer for their name, and then gets their details. Then the bot uses those details to find the right kinds of shoes for them, then checks its inventory database to be sure they're in stock, then creates a prompt that asks the FM to generate a response to the customer. Then the bot places an order on behalf of a customer." height="700" width="700" style="background-color:white; padding:1em;" /></center> <br/>

In this use-case, we use the Lambda function to retrieve customer details, list shoes matching customer preferred activity and finally, place orders. Our code is backed by an in-memory SQLite database. You can use similar constructs to write to a persistent data store. 

This lab will demonstrate how to create your first agent for Bedrock using the BOTO3 SDK:

1. Select the underlying foundation model (FM) for your agent 
2. Provide a clear and concise agent instruction 
3. Create and associate an action group with an API Schema and a Lambda function 
4. Create, invoke, test, and deploy the agent
5. Demonstrating a chat session with multi-turn conversations
6. Clean up resources (Be Frugal)

We provide a step-by-step guide with building blocks to create a customer service retail agent bot. We use a text generation model (Anthropic Claude 2) and agents for Amazon Bedrock for this solution. 

This lab notebook has the following sections:

1. <a href="#1">Environment configuration</a>
2. <a href="#2">Set up Bedrock for inference</a>
3. <a href="#3">Setup prefix variables for various agent resources</a>
4. <a href="#4">Create Lambda function for action group </a>
5. <a href="#5">Creating an agent</a>
6. <a href="#6">Deploy agent and create agent alias</a>
7. <a href="#7">Invoke agent </a>
8. <a href="#8">Multi-turn conversations / session context management</a>
9. <a href="#9"> Clean-up resources</a>
10. <a href="#10">Challenge exercise and lab quiz</a>
    
Please work top to bottom of this notebook and don't skip sections as this could lead to error messages due to missing code.

---


You will be presented with two kinds of exercises throughout the notebook: activities and challenges. <br/>

| <img style="float: center;" src="images/activity.png" alt="Activity" width="125"/>| <img style="float: center;" src="images/challenge.png" alt="Challenge" width="125"/>|
| --- | --- |
|<p style="text-align:center;"> No coding is needed for an activity. You try to understand a concept, <br/>answer questions, or run a code cell.</p> |<p style="text-align:center;">Challenges are where you can practice your coding skills.</p>


## <a name="1">Environment Setup and Configuration</a>
(<a href="#0">Go to top</a>)

Let's start by installing all required packages as specified in the `requirements.txt` file and importing several libraries.

In [1]:
%%capture
!pip3 install -r requirements.txt --quiet

In [2]:
import uuid

import pprint
import botocore
import logging
import sys
import boto3
import botocore
import json 

from mlu_utils.agents_utils import *
from mlu_utils.show_trace_widget import *
from mlu_utils.summarize_agent_trace import *

# setting up the logger config and format of log messages
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)


[2024-07-11 18:08:12,401] p29893 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [3]:

import logging
import boto3
import random
import time
import zipfile
from io import BytesIO
import json
import uuid
import pprint
import os
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
from IPython.display import Markdown
import pandas as pd
import json
import pprint

from mlu_utils.agents_infra_utils_no_kb_setup import *


In [4]:
# formatter for regular print
pp = pprint.PrettyPrinter(width=41, compact=True)

# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')


In [5]:
clean_up_trace_files("./trace_files/")

#### Session, Region and Account Setup

In [6]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '339712993987')

In [7]:
import ipywidgets as widgets
from IPython.display import JSON

out_2a_tabs_1 = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_2a_tabs_2 = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_2a_tabs_3 = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))

out_2a_turn_1_summary = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_2a_turn_2_summary = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_2a_turn_3_summary = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))


### <a name="2">2. Set up Bedrock for inference</a>
(<a href="#0">Go to top</a>)

To get started, set up Bedrock and instantiate an active `bedrock-runtime` to query LLMs. The code below leverages [LangChain's Bedrock integration](https://python.langchain.com/docs/integrations/llms/bedrock).
```
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

```
<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>A note on Bedrock API invocation</h4>

Amazon Bedrock is generally available for all AWS customers. The provided lab for this course currently invokes Bedrock's external endpoint, which has access to Anthropic and AI21 models.

For training and learning purposes, Amazon Bedrock is also available through an internal endpoint that allows accessing Amazon's Titan models. To use that instead, please follow the instructions in the <a href="https://w.amazon.com/bin/view/AmazonBedrock/Products/GetStarted/">Amazon Bedrock Get Started wiki page</a>.
</div>
</br>

In [8]:
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

### <a name="3">3. Setup prefix variables for various agent resources</a>
(<a href="#0">Go to top</a>)


This is the same set of instructions for infrastructure setup as provided in Lab 2a and includes:
- Setup for prefix variables with various agent resources
- Create Lambda function for action group
- Define SQL queries inside the Lambda function for each use-case API, followed by testing and deploying the Lambda function
- Creating an agent



In [9]:
infra_response = setup_agent_infrastructure(schema_filename='retail-agent-openapi.json', 
                                            kb_db_file_uri='retail-kb', 
                                            lambda_code_uri='lambda_retail_agent.py')


[2024-07-11 18:08:13,134] p29893 {agents_infra_utils_no_kb_setup.py:53} INFO - random_uuid :: 01147bed-02b0-49d2-a6b4-3072996ce5f6 prefix_infra :: l201147b prefix_iam :: l202b0
[2024-07-11 18:08:13,472] p29893 {agents_infra_utils_no_kb_setup.py:120} INFO - kb_db_files_path :: retail-kb kb_db_key :: kbdb_l201147b
[2024-07-11 18:08:44,462] p29893 {agents_infra_utils_no_kb_setup.py:272} INFO - bucket_name :: l201147b-agent-kb-339712993987 
 agent_name :: l201147b-agent-kb 
 agent_alias_name :: l201147b-workshop-alias 
 schema_key :: l201147b-agent-kb-schema.json  


In [10]:
agent_name = infra_response["agent_name"]
agent_alias_name = infra_response["agent_alias_name"]
agent_role = infra_response["agent_role"]
bucket_name = infra_response["bucket_name"]
schema_key = infra_response["schema_key"]
lambda_name = infra_response["lambda_name"]
lambda_function = infra_response["lambda_function"]
agent_bedrock_policy = infra_response["agent_bedrock_policy"]
agent_s3_schema_policy = infra_response["agent_s3_schema_policy"]
agent_role_name = infra_response["agent_role_name"]
lambda_role_name = infra_response["lambda_role_name"]



### <a name="5">Creating an agent</a>
(<a href="#0">Go to top</a>)


Once the needed IAM role is created, we can use the Bedrock agent client to create a new agent. To do so we use the create_agent function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

As long as the `idleSessionTTLInSeconds` time that you set in the agent configuration has not expired, you maintain the same session with the agent. 


In [11]:
# Create agent
# Haiku and Titan Premier works
# amazon.titan-text-premier-v1:0 and anthropic.claude-3-haiku-20240307-v1:0
agent_instruction = """
You are an agent that helps customers purchase shoes. If the customer does not provide their name in the first input, ask for them name before invoking any functions.
Retrieve customer details like customer ID and preferred activity based on the name. 
Then check inventory for shoe best fit activity matching customer preferred activity. 
Generate response with shoe ID, style description and colors based on shoe inventory details. 
If multiple matches exist, display all of them to the user. 
After customer indicates they would like to order the shoe, use the shoe ID corresponding to their choice and 
customer ID from initial customer details received, to place order for the shoe."""

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Retail agent for shoe purchase.",
    idleSessionTTLInSeconds=3600,
    foundationModel="anthropic.claude-3-haiku-20240307-v1:0",
    instruction=agent_instruction,
)

In [12]:
agent_id = response['agent']['agentId']
agent_id

'BIXPWQTW61'

### Create agent action group

We will now create and agent action group that uses the lambda function and API schema files created before. The create_agent_action_group function provides this functionality. We will use DRAFT as the agent version since we haven't yet create an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.


In [13]:
bucket_name, schema_key

('l201147b-agent-kb-339712993987', 'l201147b-agent-kb-schema.json')

In [14]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName='RetailManagementActionGroup',
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': schema_key
        }
    },
    description='Actions for a retail agent that helps customers purchase shoes.'
)

In [15]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': 'a2d865dc-3d67-4c09-99ce-888fbea50b71',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jul 2024 18:09:15 GMT',
   'content-type': 'application/json',
   'content-length': '572',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a2d865dc-3d67-4c09-99ce-888fbea50b71',
   'x-amz-apigw-id': 'awnnvFgDoAMEqZw=',
   'x-amzn-trace-id': 'Root=1-66901fca-0f66c911767c325d160a6716'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:339712993987:function:l201147b-agent-kb-339712993987'},
  'actionGroupId': 'DJR2UN54ES',
  'actionGroupName': 'RetailManagementActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'BIXPWQTW61',
  'agentVersion': 'DRAFT',
  'apiSchema': {'s3': {'s3BucketName': 'l201147b-agent-kb-339712993987',
    's3ObjectKey': 'l201147b-agent-kb-schema.json'}},
  'createdAt': datetime.datetime(2024, 7, 11, 18, 9, 15, 486580, tzinfo=tzlocal()),
  'description': 'Actions fo

### Allowing agent to invoke action group Lambda

Before using our action group, we need to allow our agent to invoke the Lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the Lambda function created:


In [16]:
# Create allow invoke permission on Lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

### Testing agent

Let's create a DRAFT version of the agent that can be used for internal testing.


In [17]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': '65849f80-d7da-48a5-9522-a9e879eb9f22',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 11 Jul 2024 18:09:15 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '65849f80-d7da-48a5-9522-a9e879eb9f22',
   'x-amz-apigw-id': 'awnn3GQkoAMEd5A=',
   'x-amzn-trace-id': 'Root=1-66901fcb-57068724334b4e8927f0af14'},
  'RetryAttempts': 0},
 'agentId': 'BIXPWQTW61',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 7, 11, 18, 9, 15, 764635, tzinfo=tzlocal())}

### <a name="6">Deploy agent and create agent alias</a>
(<a href="#0">Go to top</a>)

We will now create an alias of the agent that can be used to deploy the agent.


In [18]:
# Pause to make sure agent is prepared
time.sleep(10)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)
# Pause to make sure agent alias is ready
time.sleep(10)

In [19]:
agent_alias

{'ResponseMetadata': {'RequestId': '4b43fc73-4205-4229-82b6-1817a1432578',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 11 Jul 2024 18:09:25 GMT',
   'content-type': 'application/json',
   'content-length': '349',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4b43fc73-4205-4229-82b6-1817a1432578',
   'x-amz-apigw-id': 'awnpdE0DIAMEPvA=',
   'x-amzn-trace-id': 'Root=1-66901fd5-41c1c4893ff0ff7b33f0599f'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-east-1:339712993987:agent-alias/BIXPWQTW61/JUQXRTZUM9',
  'agentAliasId': 'JUQXRTZUM9',
  'agentAliasName': 'l201147b-workshop-alias',
  'agentAliasStatus': 'CREATING',
  'agentId': 'BIXPWQTW61',
  'createdAt': datetime.datetime(2024, 7, 11, 18, 9, 25, 923664, tzinfo=tzlocal()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2024, 7, 11, 18, 9, 25, 923664, tzinfo=tzlocal())}}

### <a name="7">Invoke agent</a>
(<a href="#0">Go to top</a>)

Now that we've created the agent, let's use the bedrock-agent-runtime client to invoke this agent and perform some tasks.

<b> Note: If your kernel session is active, you will need to re-run this section for any new prompts you want to try once the session is over </b>




In [20]:

# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

print(f"agent_alias_id :::: {agent_alias_id}")
print(f"agent_id :::: {agent_id}")

agent_alias_id :::: JUQXRTZUM9
agent_id :::: BIXPWQTW61


### Turn 1 : First user interaction with agent

In [21]:
%%time

# first question - turn 1
session_id:str = str(uuid.uuid1()) # random identifier
enable_trace:bool = True
end_session:bool = False
final_answer = None

print(f"session_id :::: {session_id}")
# replace this with a prompt relevant to your agent
input_text:str = "Hello, My name is John Doe. I am looking to buy running shoes" 

final_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                               input_text, 
                                               agent_id, 
                                               agent_alias_id, 
                                               session_id, 
                                               enable_trace,
                                               end_session,
                                               trace_filename_prefix = 'lab2a_agent_trace',
                                               turn_number = 1)

session_id :::: bb3fb528-3fb0-11ef-b235-16ffc4ecb111


[2024-07-11 18:09:37,391] p29893 {agents_utils.py:87} INFO - Final answer ->
Hello, could you please provide your name?


CPU times: user 20.8 ms, sys: 3.02 ms, total: 23.8 ms
Wall time: 1.38 s


In [22]:
# Print the final response for turn-1
format_final_response(question=input_text, final_answer=final_answer, lab_number="2a", turn_number="1", gen_sql=False)

,User Question,Agent Answer
0,"Hello, My name is John Doe. I am looking to buy running shoes","Hello, could you please provide your name?"


In [23]:
# Deep dive into Agent-workflow steps in each of the tab
%load_ext autoreload
%autoreload 2
from mlu_utils.show_trace_widget import *

show_tabs(trace_filename_prefix = 'lab2a_agent_trace', turn_number = 1)
display(out_2a_tabs_1)

trace_filename_prefix = lab2a_agent_trace and turn_number = 1


ToggleButtons(button_style='success', description='Agent Trace: (Last Logged trace only) Full trace available …

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [24]:
# Wait for tab output async processes to complete
#time.sleep(10)

In [25]:
%%time
# using Claude-v3 Haiku to generate a summary of the agent's workflow for this conversation turn
turn_1_summary = summarize_agent_trace(trace_file_base_path= "trace_files/", lab_number="2a", turn_number="1")
with out_2a_turn_1_summary:
    out_2a_turn_1_summary.clear_output()
    display(Markdown(turn_1_summary))


>>>>>>>> complete_log_path to summarize==> trace_files/full_trace_lab2a_agent_trace_1.log


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: user 136 ms, sys: 25.3 ms, total: 161 ms
Wall time: 3.5 s


In [26]:
## Display the summary of the agent workflow trace
display(out_2a_turn_1_summary)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

### <a name="8">Multi-turn conversations / session context management</a>
(<a href="#0">Go to top</a>)


In this section, we continue with the same user-session using the `session_id` parameter and answer and ask followup questions to the agent.
We call every round of conversation a turn, so therefore this chat session can be considered as a multi-turn conversation.


### Turn 2 : Second user interaction with agent for same session

In [27]:
%%time
# Follow-up question - turn 2
print(f"Session ID :: {session_id}")
enable_trace:bool = True
end_session:bool = False

# Replace this with a prompt relevant to your agent
input_text:str = "Can you elaborate more about Shoe ID 10?" 

final_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                               input_text, 
                                               agent_id, 
                                               agent_alias_id, 
                                               session_id, 
                                               enable_trace,
                                               end_session,
                                               trace_filename_prefix = 'lab2a_agent_trace',
                                               turn_number = 2)

Session ID :: bb3fb528-3fb0-11ef-b235-16ffc4ecb111


[2024-07-11 18:09:49,088] p29893 {agents_utils.py:87} INFO - Final answer ->
The Shoe ID 10 are lightweight racing flats designed for running. They have a breathable mesh upper, responsive cushioning, and a flexible outsole for a fast, efficient ride. The shoes come in bright yellow and orange color options and are priced at $129.99. The current inventory count for this shoe is 912 units.


CPU times: user 17.4 ms, sys: 606 µs, total: 18 ms
Wall time: 7.25 s


In [28]:
# Print the final response for turn-2
format_final_response(question=input_text, final_answer=final_answer, lab_number="2a", turn_number="2", gen_sql=False)

,User Question,Agent Answer
0,Can you elaborate more about Shoe ID 10?,"The Shoe ID 10 are lightweight racing flats designed for running. They have a breathable mesh upper, responsive cushioning, and a flexible outsole for a fast, efficient ride. The shoes come in bright yellow and orange color options and are priced at \$129.99. The current inventory count for this shoe is 912 units."


In [29]:
# Deep dive into agent-workflow steps in each of the tab
%load_ext autoreload
%autoreload 2
from mlu_utils.show_trace_widget import *

show_tabs(trace_filename_prefix = 'lab2a_agent_trace', turn_number = 2)
display(out_2a_tabs_2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
trace_filename_prefix = lab2a_agent_trace and turn_number = 2


ToggleButtons(button_style='success', description='Agent Trace: (Last Logged trace only) Full trace available …

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [30]:
# Wait for tab output async processes to complete
time.sleep(10)

In [31]:
%%time
# Using Claude-v3 Haiku to generate a summary of the agent's workflow for this conversation turn
turn_2_summary = summarize_agent_trace(trace_file_base_path= "trace_files/", lab_number="2a", turn_number="2")
with out_2a_turn_2_summary:
    out_2a_turn_2_summary.clear_output()
    display(Markdown(turn_2_summary))


>>>>>>>> complete_log_path to summarize==> trace_files/full_trace_lab2a_agent_trace_2.log
CPU times: user 12.3 ms, sys: 315 µs, total: 12.6 ms
Wall time: 3.65 s


In [32]:
## Display the summary of the agent workflow trace
display(out_2a_turn_2_summary)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

### Turn 3 : Third user interaction with agent for same session

In [33]:
%%time
# Follow-up question - turn 3
print(f"Session ID :: {session_id}")
enable_trace:bool = True
end_session:bool = False

# Replace this with a prompt relevant to your agent
input_text:str = "Place order for shoe ID 10" 

final_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                               input_text, 
                                               agent_id, 
                                               agent_alias_id, 
                                               session_id, 
                                               enable_trace,
                                               end_session,
                                               trace_filename_prefix = 'lab2a_agent_trace',
                                               turn_number = 3)

Session ID :: bb3fb528-3fb0-11ef-b235-16ffc4ecb111


[2024-07-11 18:10:09,198] p29893 {agents_utils.py:87} INFO - Final answer ->
I have placed the order for Shoe ID 10 for you, John Doe. The order has been processed and recorded in our system. Please let me know if you need anything else!


CPU times: user 21.7 ms, sys: 76 µs, total: 21.8 ms
Wall time: 6.15 s


In [34]:
# Print the final response for turn-3
format_final_response(question=input_text, final_answer=final_answer, lab_number="2a", turn_number="3", gen_sql=False)

,User Question,Agent Answer
0,Place order for shoe ID 10,"I have placed the order for Shoe ID 10 for you, John Doe. The order has been processed and recorded in our system. Please let me know if you need anything else!"


In [35]:
# Deep dive into Agent-workflow steps in each of the tab
%load_ext autoreload
%autoreload 2
from mlu_utils.show_trace_widget import *

show_tabs(trace_filename_prefix = 'lab2a_agent_trace', turn_number = 3)
display(out_2a_tabs_3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
trace_filename_prefix = lab2a_agent_trace and turn_number = 3


ToggleButtons(button_style='success', description='Agent Trace: (Last Logged trace only) Full trace available …

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [36]:
# Wait for tab output async processes to complete
time.sleep(10)

In [37]:
%%time
# Using Claude-v3 Haiku to generate a summary of the agent's workflow for this conversation turn
turn_3_summary = summarize_agent_trace(trace_file_base_path= "trace_files/", lab_number="2a", turn_number="3")
with out_2a_turn_3_summary:
    out_2a_turn_3_summary.clear_output()
    display(Markdown(turn_3_summary))


>>>>>>>> complete_log_path to summarize==> trace_files/full_trace_lab2a_agent_trace_3.log
CPU times: user 13.2 ms, sys: 160 µs, total: 13.4 ms
Wall time: 3.52 s


In [38]:
%%time
## Display the summary of the agent workflow trace
display(out_2a_turn_3_summary)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

CPU times: user 2.51 ms, sys: 0 ns, total: 2.51 ms
Wall time: 2.75 ms


### <a name="9">[Be Frugal] Clean up resources </a>
(<a href="#0">Go to top</a>)


##### In the following cell, we keep an option to raise an exception to avoid auto-executing the next block of lines and optionally cleanup all resources. This is useful when the `Kernel > run all` option is used.

`Please be frugal if you choose to enable this exception in the code cell below. By default it is disabled and all resources will be cleaned up immediately to avoid additional costs.`

##### Within the same kernel session, this will allow experimentation with different prompts without having to recreate agent resources (takes ~5 minutes)

In [39]:
# This avoids auto-cleanup
Raise Exception('Avoiding Auto-Cleanup of Bedrock Agent Resources')


SyntaxError: invalid syntax (3470409141.py, line 2)

In [ ]:
cleanup_infrastructure(agent_action_group_response, lambda_name, lambda_function, lambda_role_name, agent_id, agent_alias_id, agent_role_name, bucket_name, schema_key, agent_bedrock_policy, agent_s3_schema_policy)

### <a name="10">Challenge Exercise and Lab Quiz </a>
(<a href="#0">Go to top</a>)

### Challenge Exercise 

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <h2><i>Try it Yourself!</i></h2>
    <br>
    <p style="text-align:center;margin:auto;"><img src="./images/challenge.png" alt="Challenge" width="100" /> </p>
    <p style=" text-align: center; margin: auto;">Try the following exercise to harness the power of bedrock agents uptil now:</p>
    <br>
</div>


- Try a new set of conversations/prompts to buy another pair of shoes for the same customer and list inventory
- Try adding more customer information in the customer retail SQLite DB like maybe your name and preference of shoes and use that in your prompts

---

### Lab Quiz

Well done on completing the lab! Now, it's time for a brief knowledge assessment.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <h2><i>Try it Yourself!</i></h2>
    <br>
    <p style="text-align:center;margin:auto;"><img src="./images/activity.png" alt="Challenge" width="100" /> </p>
    <p style=" text-align: center; margin: auto;">Answer the following questions to test your understanding of bedrock agents uptil now.</p>
    <br>
</div>


In [ ]:
from mlu_utils.agents_quiz import *

lab2a_question1

In [ ]:
lab2a_question2